In [74]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt


In [75]:
# import and drop duplicate
df = pd.read_csv('data/googleplaystore.csv', ).drop_duplicates()

# rename columns to lower case, underscored space
df.columns = [col.lower().replace(' ', '_') for col in df.columns]

# drop version columns, won't be needed in regression
df = df.iloc[:, :-2]

In [76]:
df.head(1).T

,0
app,Photo Editor & Candy Camera & Grid & ScrapBook
category,ART_AND_DESIGN
rating,4.1
reviews,159
size,19M
installs,"10,000+"
type,Free
price,0
content_rating,Everyone
genres,Art & Design


In [77]:
# find any null rows
null_cols = df.isna().any()[df.isna().any().values] # just drop them

# drop null rows
df = df.dropna()

In [78]:
# clean column App, remove duplicates
dup_apps = df.app.value_counts()[df.app.value_counts() > 1].index
print('Duplicate app names: ', dup_apps)
df = df.sort_values(by='last_updated').drop_duplicates('app', keep='last')

Duplicate app names:  Index(['ROBLOX', '8 Ball Pool', 'Zombie Catchers', 'Helix Jump',
       'Bubble Shooter', 'Candy Crush Saga', 'Angry Birds Classic',
       'Subway Surfers', 'slither.io', 'Bowmasters',
       ...
       'letgo: Buy & Sell Used Stuff, Cars & Real Estate',
       'Zomato - Restaurant Finder and Food Delivery App',
       'Crunchyroll - Everything Anime', 'G Cloud Backup',
       'OfficeSuite : Free Office + PDF Editor', 'Flashlight',
       'Just She - Top Lesbian Dating',
       'Groupon - Shop Deals, Discounts & Coupons',
       'Mapy.cz - Cycling & Hiking offline maps',
       'AccuWeather: Daily Forecast & Live Weather Reports'],
      dtype='object', length=521)


In [79]:
# standardize size column and conver to float

def covert2mb(val):
    val = val.lower()
    unit = val[-1]
    if 'm' == unit:
        return float(val.replace('m',''))
    elif 'k' == unit:
        return float(val.replace('k','')) / 1024
    else:
        return 0  # to be removed from df

df['app_size'] = df['size'].apply(covert2mb)


In [80]:
# remove size 'varies by device'
df = df[df['app_size']>0]
# drop size column
df = df.drop(columns='size')

In [81]:
# convert installs from interval to integer
df.installs = df.installs.str.replace('[\,|\+]+', '').astype('int')

In [82]:
# convert reviews to integer
df.reviews = df.reviews.astype('int')

In [83]:
# convert price to float
df[df.price != '0'].price.value_counts()
df['price'] = df.price.str.replace('$','').astype(float)

In [84]:
# clean cateogry
df.category.value_counts()
df.genres.value_counts().head(50)

df['category'] = df['category'].str.title()


In [85]:
# encode last_updated date to number

from datetime import datetime

df['last_updated'] = pd.to_datetime(df.last_updated)
def date2num(val):
  start = datetime(2010,1,1)
  diff = val - start
  return diff.days

df['last_updated_encoded'] = df.last_updated.apply(date2num)


In [86]:
df.last_updated_encoded.describe()
df = df.drop(columns='last_updated')

In [87]:
# drop genre and name because too many categories
df = df.drop(columns=['app', 'genres'])

In [88]:
# export initial data
df.head()
df.to_csv('data/data_initial.csv', index=False)

In [89]:
# log transform dataset
df[['installs','reviews']] = np.log(df[['installs','reviews']])

In [90]:
# lump categories together
category_lump = {'Tools':[ 'Auto_And_Vehicles',  'Weather', 'Tools',  'Productivity', 'Personalization', 'Medical',  'Maps_And_Navigation',  'Libraries_And_Demo']
           ,'Entertainment':['Video_Players', 'Sports', 'Photography', 'Entertainment',  'Game','Art_And_Design', 'Comics', 'Events']
           ,'Lifestyle': ['Health_And_Fitness', 'Beauty','Travel_And_Local','Shopping','Lifestyle','Parenting','Food_And_Drink' ,'Family','House_And_Home',  'Libraries_And_Demo']
           ,'Social':['Social','Dating', 'Communication']
           ,'Education/Business':[ 'Books_And_Reference', 'Business', 'Education', 'News_And_Magazines','Finance']
           }
cat_assign = {cat: key for key, cat_ls in category_lump.items() for cat in cat_ls}

df['category'] = df.category.apply(lambda x : cat_assign[x])

In [91]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2,random_state=1)
train.to_csv('data/train_alltype.csv', index=False)
test.to_csv('data/test_alltype.csv', index=False)

In [92]:
# filter data down to paid apps only
paid = df[(df.type == 'Paid')].copy()

#log transform price
paid['price'] = np.log(paid['price'])

train, test = train_test_split(paid, test_size=0.2,random_state=1)
train.to_csv('data/train.csv', index=False)
test.to_csv('data/test.csv', index=False)